## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')


## Data Loading and pre-processing

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

def load_adherer_data():
    """
    Load medication events data from AdhereR package
    """
    # Activate pandas-R conversion
    pandas2ri.activate()
    
    # Import AdhereR package
    adherer = importr('AdhereR')
    
    # Get med.events dataset
    med_events = robjects.r('med.events')
    
    # Convert to pandas DataFrame
    df = pandas2ri.rpy2py(med_events)
    
    # Convert column names to match the example
    df.columns = ['pnr', 'eksd', 'perday', 'ATC', 'dur_original']
    
    # Convert date column
    df['eksd'] = pd.to_datetime(df['eksd'])
    
    return df

def analyze_medication(data, medication_code):
    """
    Analyze a specific medication using Sessa's method
    """
    # Filter for specific medication
    med_data = data[data['ATC'] == medication_code].copy()
    
    # Sort by patient and date
    med_data = med_data.sort_values(['pnr', 'eksd'])
    
    # Calculate intervals between prescriptions
    med_data['prev_eksd'] = med_data.groupby('pnr')['eksd'].shift(1)
    med_data['event_interval'] = (med_data['eksd'] - med_data['prev_eksd']).dt.days
    
    # Remove rows with NA intervals
    med_data = med_data.dropna(subset=['event_interval'])
    
    # Calculate ECDF
    sorted_intervals = np.sort(med_data['event_interval'])
    ecdf = np.arange(1, len(sorted_intervals) + 1) / len(sorted_intervals)
    
    # Keep only lower 80% of ECDF
    mask = ecdf <= 0.8
    filtered_intervals = sorted_intervals[mask]
    
    return med_data, filtered_intervals

def cluster_intervals(intervals, method='kmeans'):
    """
    Cluster the intervals using either K-means or DBSCAN
    """
    # Scale the data
    X = StandardScaler().fit_transform(intervals.reshape(-1, 1))
    
    if method == 'kmeans':
        # Find optimal number of clusters using silhouette score
        best_score = -1
        best_k = 2
        for k in range(2, min(10, len(intervals))):
            kmeans = KMeans(n_clusters=k, random_state=42)
            labels = kmeans.fit_predict(X)
            score = silhouette_score(X, labels)
            if score > best_score:
                best_score = score
                best_k = k
        
        # Perform final clustering
        kmeans = KMeans(n_clusters=best_k, random_state=42)
        labels = kmeans.fit_predict(X)
        
    else:  # DBSCAN
        dbscan = DBSCAN(eps=0.5, min_samples=5)
        labels = dbscan.fit_predict(X)
    
    return labels

def main_analysis():
    """
    Main analysis function
    """
    # Load AdhereR data
    print("Loading AdhereR data...")
    data = load_adherer_data()
    
    # Get unique medications
    medications = data['ATC'].unique()
    print(f"Found {len(medications)} unique medications")
    
    # Analyze first two medications
    for med in medications[:2]:
        print(f"\nAnalyzing medication {med}")
        
        # Get processed data
        med_data, filtered_intervals = analyze_medication(data, med)
        
        # Perform both clustering methods
        kmeans_labels = cluster_intervals(filtered_intervals, 'kmeans')
        dbscan_labels = cluster_intervals(filtered_intervals, 'dbscan')
        
        # Create visualizations
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot original distribution
        sns.histplot(data=med_data, x='event_interval', ax=ax1)
        ax1.set_title(f'Distribution of intervals for {med}')
        
        # Plot ECDF
        sorted_intervals = np.sort(med_data['event_interval'])
        ecdf = np.arange(1, len(sorted_intervals) + 1) / len(sorted_intervals)
        ax2.plot(sorted_intervals, ecdf)
        ax2.axhline(y=0.8, color='r', linestyle='--')
        ax2.set_title('ECDF with 80% cutoff')
        
        # Plot K-means clusters
        for cluster in np.unique(kmeans_labels):
            cluster_data = filtered_intervals[kmeans_labels == cluster]
            sns.kdeplot(data=cluster_data, ax=ax3)
        ax3.set_title('K-means clustering')
        
        # Plot DBSCAN clusters
        for cluster in np.unique(dbscan_labels):
            if cluster != -1:  # Skip noise points
                cluster_data = filtered_intervals[dbscan_labels == cluster]
                sns.kdeplot(data=cluster_data, ax=ax4)
        ax4.set_title('DBSCAN clustering')
        
        plt.tight_layout()
        plt.savefig(f'medication_{med}_analysis.png')
        plt.close()
        
        # Calculate statistics for each cluster
        for method, labels in [('K-means', kmeans_labels), ('DBSCAN', dbscan_labels)]:
            print(f"\n{method} clustering results:")
            for cluster in np.unique(labels):
                if cluster != -1:  # Skip noise points for DBSCAN
                    cluster_data = filtered_intervals[labels == cluster]
                    print(f"Cluster {cluster}:")
                    print(f"  Size: {len(cluster_data)}")
                    print(f"  Median interval: {np.median(cluster_data):.1f} days")
                    print(f"  Mean interval: {np.mean(cluster_data):.1f} days")
                    print(f"  Std deviation: {np.std(cluster_data):.1f} days")

if __name__ == "__main__":
    main_analysis()

ModuleNotFoundError: No module named 'rpy2'